# Neural network for regression tasks
**here I just tested a neural for regression task, bellow we will find the used datasets**

Airfoil Self-Noise : https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise

In [ ]:
import numpy
import plotly
import pandas

In [2]:
class Sigmoid:
    def function(data):
        return 1 / (1 + numpy.exp(-data))
    
    def derivative(data):
        return Sigmoid.function(data) - Sigmoid.function(data)**2

class ReLU:
    def function(data):
        return numpy.maximum(data, 0)
    def derivative(data):
        return (data > 0) * 1

class Mse:
    # maybe to change
    def function(data, expected):
        return (1 / data.shape[0]) * numpy.sum((expected - data)**2)
        

    def derivative(y, predicted):
        return predicted - y

class NeuralNet:
    def __init__(self, layers, learning_rate, a_fct, epochs, minibatch=None):
        self.nets = []
        self.biases = []
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.a_fct = a_fct
        self.minibatch = minibatch
        layers = zip(layers, layers[1:])
        for i in layers:
            net = numpy.random.rand(i[1],i[0])
            b = numpy.random.rand(i[1])
            self.nets.append(net)
            self.biases.append(b)

    def feed_forward(self, data, training=False):
        a = [data]
        for i in range(0, len(self.nets)):
            data = numpy.dot(data, self.nets[i].T) + self.biases[i]
            if training:
                a.append(data)
            data = self.a_fct.function(data)
        if training:
            return a, data
        return data
    
    def evaluate(self, test_data):
        error = 0
        for ipt, expected in test_data:
            output = self.feed_forward(ipt)
            error += Mse.function(output, expected)
        return error / len(test_data)


    def train_sample(self, data):
        delta_b = [ numpy.zeros(self.biases[i].shape) for i in range(len(self.nets))]
        delta_w = [ numpy.zeros(self.nets[i].shape) for i in range(len(self.nets))]
        for feed,expected in data:
            a, data = self.feed_forward(feed, training=True)
            
            # first delta
            delta = Mse.derivative(expected, data) * self.a_fct.derivative(a[-1])
            
            delta_w[-1] += numpy.outer(delta, self.a_fct.function(a[-2]))
            delta_b[-1] += delta
            
            next_w_index = len(self.nets) - 2
            next_layer_index = (len(self.nets) + 1) - 2
            
            for i in range(next_w_index, -1, -1):
                delta = numpy.dot(self.nets[i + 1].T, delta) * self.a_fct.derivative(a[next_layer_index])
                delta_b[i] += delta
                if (next_layer_index - 1) == 0:
                    delta_w[i] += numpy.outer(delta, a[next_layer_index - 1])
                else:
                    delta_w[i] += numpy.outer(delta, self.a_fct.function(a[next_layer_index - 1]))
                next_layer_index = next_layer_index - 1
                
        return delta_b, delta_w

    def train(self, train_data, test_data):
        error_arr = [] 
        for i in range(0, self.epochs):
            self.mini_batch(train_data, 1/5)

            # return error rate
            error_rate = self.evaluate(test_data)
            error_arr.append(error_rate)
        return error_arr

    # decided to use the mini batch gd because it is a compromise between sgd(change gradient for each training data input) and bgd(change gradient after getting gradient from all train dataset)
    def mini_batch(self, train_data, minib_portion): # portion between 0 and 1
        numpy.random.shuffle(train_data)
        nb_of_input = len(train_data)
        k = self.minibatch
        if k == None:
            k = int(nb_of_input * minib_portion)

        for i in range(k, nb_of_input, k):
            
            delta_b, delta_w = self.train_sample(train_data[i - k:i])
            for i in range(len(self.biases)):
                self.biases[i] = self.biases[i] - self.learning_rate * delta_b[i] / k
                self.nets[i] = self.nets[i] - self.learning_rate * delta_w[i] / k
        # epoch finished

    # just to test the gd
    def with_one_sample(self, sample):
        # training 
        (feed, expected) = sample
        print("feed :", feed)
        delta_b = [ numpy.zeros(self.biases[i].shape) for i in range(len(self.nets))]
        delta_w = [ numpy.zeros(self.nets[i].shape) for i in range(len(self.nets))]
        a, data = self.feed_forward(feed, training=True)

        # first delta
        
        print(data)
        print(expected)
        
        delta = Mse.derivative(expected, data) * self.a_fct.derivative(a[-1])

        delta_w[-1] = numpy.outer(delta, self.a_fct.function(a[-2]))
        delta_b[-1] = delta

        next_w_index = len(self.nets) - 2
        next_layer_index = (len(self.nets) + 1) - 2

        
        for i in range(next_w_index, -1, -1):
            delta = numpy.dot(self.nets[i + 1].T, delta) * self.a_fct.derivative(a[next_layer_index])
            delta_b[i] = delta
            if (next_layer_index - 1) == 0:
                delta_w[i] = numpy.outer(delta, a[next_layer_index - 1])
            else:
                delta_w[i] = numpy.outer(delta, self.a_fct.function(a[next_layer_index - 1]))
            next_layer_index = next_layer_index - 1
        
        for i in range(len(self.biases)):
            self.biases[i]  -= self.learning_rate * delta_b[i]
            self.nets[i] -= self.learning_rate * delta_w[i]

        output = self.feed_forward(feed)
        error = numpy.sum(Mse.function(output, expected))
        
        return error
    
# data scalling method: needed so that each variable is "considered" equally
# the downside of normalization is if in your data there is outsiders the results will be biased, on the other hand
# if I used standardization I will assumed that the given data is normally distributed

def normalization(data):
    data_min = numpy.min(data)
    data_max = numpy.max(data)
    return (data - data_min) / (data_max - data_min)


In [10]:
# convert into csv tmls
import random
data = pandas.read_csv("./airfoil_self_noise.dat", header=None, sep="\t")
#data = pandas.read_csv("./airfoil_self_noise.dat", skiprows=[1]).rename(columns={"class": "c"})
data

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [11]:
data.to_csv("airfoil_self_noise.csv", index=False) # remove index from table

In [16]:
data = pandas.read_csv("./airfoil_self_noise.tmls", skiprows=[1]).rename(columns={"sound pressure level": "c"})

# normalize data
varbls = [i for i in data.columns.to_list() if i != "c"]

for v in varbls:
    data[v] = normalization(data[v].to_numpy())

inp = data.loc[:, data.columns!='c'].to_numpy()   
expected = data.loc[:, data.columns=='c'].to_numpy()

In [13]:
# used to decide nb of hidden layers and nb of perceptron per hidden layer:
# https://www.kdnuggets.com/2019/11/designing-neural-networks.html

In [17]:
expected

array([[126.201],
       [125.201],
       [125.951],
       ...,
       [106.604],
       [106.224],
       [104.204]])

In [19]:
print(len(expected))

1503


In [21]:
traindata_edge = int(len(expected) * 0.8)

In [23]:
dataset = zip(inp, expected)
a = list(dataset)

random.shuffle(a)

train_data = a[:traindata_edge] # %80
test_data = a[traindata_edge:] # %20
len(varbls)

5

In [38]:
import plotly.express as px

nn1 = NeuralNet([5, 5, 5, 1], 0.0005, ReLU, 100)

fig = px.line(nn1.train(train_data, test_data), title='Loss')

fig.show()

In [39]:
print(test_data[0][0], test_data[0][1])
    
print(f"res = {nn1.feed_forward(test_data[0][0])}, expected ={test_data[0][1]}")

[0.09090909 0.14864865 0.27272727 1.         0.02805586] [134.034]
res = [130.92056829], expected =[134.034]


In [40]:
print(f"res = {nn1.feed_forward(a[1000][0])}, expected ={a[1000][1]}")

res = [125.93530854], expected =[126.54]


In [41]:
print(f"res = {nn1.feed_forward(a[60][0])}, expected ={a[60][1]}")

res = [126.282756], expected =[130.05]


In [42]:
print(f"res = {nn1.feed_forward(a[1502][0])}, expected ={a[1502][1]}")

res = [116.34707907], expected =[123.514]


In [43]:

print(f"res = {nn1.feed_forward(inp[1502])}, expected ={expected[1502]}")

res = [110.29156629], expected =[104.204]
